In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import ifft,fft
from scipy.linalg import eigh
import random
from networkx.generators.community import stochastic_block_model
from networkx.linalg.graphmatrix import adjacency_matrix
from numpy.linalg import norm
from matplotlib import cm
import time
# set random seeds
# random.seed(11)
# np.random.seed(11)

In [2]:
class SFSC_ContAng:

    def __init__(self, size_of_clusters, probs_of_sbm, num_of_clusters):

        self.graph = np.asarray(adjacency_matrix(stochastic_block_model(size_of_clusters, probs_of_sbm)).todense())
        self.true_angles_by_cluster = dict()
        self.size_of_clusters = size_of_clusters
        self.N = size_of_clusters.sum()
        self.num_of_clusters = num_of_clusters
        self.alpha_struc = None
        self.A_k = None
        self.Phi = None
        self.Pi = np.eye(self.N)
        self.cluster_id = None
        self.estimated_angles = None
        self.Rf = None

        
    def generate_angles(self):

        self.alpha_struc = np.random.uniform(low=0, high=2*np.pi, size=(self.N, self.N))
        self.alpha_struc[np.tril_indices(self.N)] = 0
        self.alpha_struc = self.alpha_struc + np.mod(-self.alpha_struc.T, 2*np.pi)
        start_idx, end_idx = 0, 0
        for idx, b_size in enumerate(self.size_of_clusters):
            end_idx += b_size
            alphas = np.random.uniform(low=0, high=2*np.pi, size=b_size)
            self.true_angles_by_cluster.setdefault(idx, alphas)
            self.alpha_struc[start_idx:end_idx, start_idx:end_idx] = np.mod(
                alphas.reshape(-1, 1) - alphas,
                2*np.pi
            )
            start_idx += b_size

        return None
    
    def construct_A_k(self):

        self.A_k = self.graph * np.exp(1j  * self.alpha_struc)
        _, eigen_vec = eigh(
            self.A_k,
            subset_by_index=[self.N - self.num_of_clusters, self.N - 1]
        )
        self.Phi = eigen_vec.T
        
        return None
    
    def cpqr(self):

        R = self.Phi.copy()
        for m in range(self.num_of_clusters):
            norm_sum = norm(R[m:, m:], axis=0)
            piv = np.argmax(norm_sum) + m
            R[:, [piv, m]] = R[:, [m, piv]]
            self.Pi[:, [piv, m]] = self.Pi[:, [m, piv]]
            _, tRm = SFSC_ContAng.qr_method_one_step(R[m:, m:])
            R[m:, m:] = tRm

        self.Rf = R @ self.Pi.T
        self.cluster_id = np.argmax(np.abs(self.Rf), axis=0)
        self.estimated_angles = np.angle(self.Rf[self.cluster_id, np.array(range(self.N))])

        return None

    @staticmethod
    def qr_method_one_step(X):

        m, _ = X.shape
        r = X[:, 0]
        alpha = -np.exp(1j * np.angle(r[0])) * norm(r)
        e1 = np.zeros(m)
        e1[0] = 1
        u = r - alpha * e1
        v = u / norm(u)
        Q = np.eye(m) - 2 * v.reshape(-1, 1) @ np.conjugate(v).reshape(1, -1)
        R = Q @ X
        R[0, :] = np.exp(-1j * np.angle(R[0, 0])) * R[0, :]

        return Q, R

In [3]:
n_sample = 20
b_sizes = np.array([500, 500])
n = b_sizes.sum()
M = 2
success_rate = np.zeros((41, 16))
angle_error = np.zeros((41, 16))

In [4]:
for i in range(2, 16):
    for j in range(2, 41):
        start = time.perf_counter()
        for t in range(n_sample):
            p = i * np.log(n) / n
            q = j * np.log(n) / n
            sbm_probs = q * np.ones((M, M)) + (p - q) * np.eye(M)
            sfsc = SFSC_ContAng(
                size_of_clusters=b_sizes,
                probs_of_sbm=sbm_probs,
                num_of_clusters=M
            )
            sfsc.generate_angles()
            sfsc.construct_A_k()
            sfsc.cpqr()

            if (sfsc.cluster_id[:b_sizes[0]].sum() == 0 and sfsc.cluster_id[b_sizes[0]:].sum() == b_sizes[1]) \
                    or (
                    sfsc.cluster_id[:b_sizes[0]].sum() == b_sizes[0] and sfsc.cluster_id[b_sizes[0]:].sum() == 0):
                success_rate[j, i] += 1

            angle_error_cluster = np.zeros(M)
            for m in range(M):
                tmp1 = np.exp(-1j*sfsc.estimated_angles[m * 500:(m + 1) * 500])
                tmp2 = np.exp(1j*sfsc.true_angles_by_cluster[m])
                d_ang = np.angle((tmp1*tmp2).mean())
                angle_diff = np.abs(np.mod(sfsc.estimated_angles[m * 500:(m + 1) * 500] + d_ang, 2*np.pi)
                                    - sfsc.true_angles_by_cluster[m])
                angle_error_cluster[m] = np.min(np.vstack((angle_diff, 2*np.pi - angle_diff)), axis=0).max()

#                 angle_estimate = sfsc.estimated_angles[m * 500:(m + 1) * 500]
#                 angle_true = sfsc.true_angles_by_cluster[m]
#                 tmp = np.zeros(360)
#                 for freq in range(360):
#                     angle_diff = np.abs(np.mod(angle_estimate + freq*np.pi/180, 2*np.pi) - angle_true)
#                     tmp[freq] = np.min(np.vstack((angle_diff, 2*np.pi - angle_diff)), axis=0).max()
#                 angle_error_cluster[m] = np.min(tmp)

            angle_error[j, i] += np.max(angle_error_cluster)

        print(i, j, angle_error[j, i] / n_sample, success_rate[j, i] / n_sample)
        print("cost %s second" % (time.perf_counter() - start))

8 2 0.039296690543115756 1.0
cost 8.0738166 second
8 3 0.05398661447504991 1.0
cost 8.169913600000001 second
8 4 0.06592409830001347 1.0
cost 8.5178701 second
8 5 0.09364581420810711 1.0
cost 8.5658243 second
8 6 0.0995338081004054 1.0
cost 8.620647400000003 second
8 7 0.19244283349587823 0.9
cost 8.9191012 second


KeyboardInterrupt: 

In [ ]:
plt.figure()
plt.imshow(success_rate[2:41, 2:16][::-1, :] / n_sample, aspect='0.3', extent=[2, 15, 2, 40], cmap='gray')
plt.xlabel(r'$\alpha$')
plt.ylabel(r'$\beta$')
plt.colorbar()
plt.grid(linestyle = '--')
plt.show()
plt.savefig("exact_recovery_SFCPQR_ContAng")

In [ ]:
initial_cmap = cm.get_cmap('gray')
reversed_cmap=initial_cmap.reversed()

plt.figure()
plt.imshow(angle_error[2:41, 2:16][::-1, :] / n_sample, aspect='0.3', extent=[2, 15, 2, 40], cmap=reversed_cmap)
plt.xlabel(r'$\alpha$')
plt.ylabel(r'$\beta$')
plt.colorbar()
plt.grid(linestyle = '--')
plt.show()
plt.savefig("angle_recovery_SFCPQR_ContAng")